In [ ]:
import os
import os.path
import numpy as np
import mxnet as mx

In [ ]:
epoch = 30
batch_size = 120
model_path = 'model'
model_name = 'mxnet_cnn_digits'
data_folder = "./dataset"

In [ ]:
def show_data(data, label, label_type, num=15):
    %matplotlib inline
    import matplotlib.pyplot as plt
    for i in range(num):
        plt.subplot(1,num,i+1)
        plt.imshow(data[i][0], cmap='Greys_r')
        plt.axis('off')
    plt.show()
    print(label_type + ' label: %s' % (label[0:num],))

In [ ]:
def load_data(path, filename):
    data_file = open(path + '/'+filename, 'r')
    data_list = data_file.readlines()
    data_file.close()
    print(str(len(data_list)-1) + ' samples loaded from ' + path + '/'+filename)
    labels = np.empty([len(data_list),])
    pixels = np.empty([len(data_list),1,28,28])
    index=0
    for record in data_list:
        all_values = record.split(',')
        if len(all_values) == 785:
            labels[index] = all_values[0]
            pixel = np.asfarray(all_values[1:]).reshape(28,28).astype(np.float32)/255
            pixels[index] = pixel
            index = index + 1  
    data = {'label': labels, 'data': pixels}    
    return data

In [ ]:
def load_model(model_path, model_name, batch_size, epoch): 
    model = mx.mod.Module.load(model_path + '/' + str(epoch) + '/batch-'+str(batch_size) + '/' + model_name, epoch, load_optimizer_states =True, context=mx.gpu())
    print('Training model '+model_name+' loaded from ' + model_path + '/' + str(epoch) + '/batch-'+str(batch_size))
    return model

In [ ]:
def measure_test_accuracy(model, test_data, batch_size, num=15):    
    test_iter = mx.io.NDArrayIter(test_data['data'], test_data['label'], batch_size)
    model.bind(data_shapes=test_iter.provide_data, label_shapes=test_iter.provide_label)
    predictions = model.predict(test_iter)
    predicted_labels = np.argmax(predictions, axis=1)
    show_data(test_data['data'],predicted_labels,"Predicted",num)
    given_labels = test_data['label']
    match = 0
    mismatch = 0
    for i,given_label in enumerate(given_labels):
        if given_label == predicted_labels[i]:
            match = match + 1
        else:
            mismatch = mismatch + 1
    return format(format(match*100/(match+mismatch),'.4f'))

In [ ]:
test_data = load_data('dataset', 'own_data_1.5.csv')
#test_data = load_data('dataset', 'validation-data.csv')
show_data(test_data['data'],test_data['label'],"Expected",15)
model = load_model(model_path, model_name, batch_size, epoch)
accuracy = measure_test_accuracy(model, test_data, batch_size)
print(accuracy)